In [68]:
!pip install scikeras 


In [92]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.utils import plot_model
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import confusion_matrix, classification_report




## Data loading

In [70]:
df = pd.read_csv("/kaggle/input/titanicdataset-traincsv/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Feature Selection

In [71]:
df = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']]
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,3,male,22.0,1,0,7.2500,0
1,1,female,38.0,1,0,71.2833,1
2,3,female,26.0,0,0,7.9250,1
3,1,female,35.0,1,0,53.1000,1
4,3,male,35.0,0,0,8.0500,0


## Preprocessing

In [72]:
df.isna().sum() / len(df)*100

Pclass       0.00000
Sex          0.00000
Age         19.86532
SibSp        0.00000
Parch        0.00000
Fare         0.00000
Survived     0.00000
dtype: float64

In [73]:
df['Age'].fillna(df['Age'].median(),inplace=True)
df.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Survived    0
dtype: int64

In [74]:
df['Sex'] = LabelEncoder().fit_transform(df['Sex'])
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,3,1,22.0,1,0,7.2500,0
1,1,0,38.0,1,0,71.2833,1
2,3,0,26.0,0,0,7.9250,1
3,1,0,35.0,1,0,53.1000,1
4,3,1,35.0,0,0,8.0500,0


In [75]:
# split features and labels
X = df.drop(columns=['Survived'], axis=1)
y = df['Survived']

In [76]:
# Train_test split 

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [77]:
def model_seq():
    model = Sequential()
    model.add(Input(shape=(X.shape[1],)))
    model.add(Dense(36, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

## Wrap the model and build pipeline

In [82]:
nn_model =KerasClassifier(model=model_seq,epochs=36, batch_size=16, validation_split=0.2)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("nn", nn_model)
])

In [83]:
pipe.fit(X_train, y_train)

Epoch 1/36
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.6229 - loss: 0.6894 - val_accuracy: 0.7203 - val_loss: 0.5895
Epoch 2/36
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7235 - loss: 0.5858 - val_accuracy: 0.8112 - val_loss: 0.5031
Epoch 3/36
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8010 - loss: 0.5194 - val_accuracy: 0.8112 - val_loss: 0.4588
Epoch 4/36
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7862 - loss: 0.4931 - val_accuracy: 0.8252 - val_loss: 0.4411
Epoch 5/36
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8222 - loss: 0.4684 - val_accuracy: 0.8182 - val_loss: 0.4259
Epoch 6/36
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8167 - loss: 0.4465 - val_accuracy: 0.8182 - val_loss: 0.4269
Epoch 7/36
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8061 - loss: 0.4521 - val_accuracy: 0.8182 - val_loss: 0.4189
Epoch 8/36
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8205 - loss: 0.4329 - val_accuracy: 0.8182 - val_loss

Pipeline(steps=[('scaler', StandardScaler()),
                ('nn',
                 KerasClassifier(batch_size=16, epochs=36, model=<function model_seq at 0x7d4c90a6c900>, validation_split=0.2))])

In [85]:
pred = pipe.predict(X_test)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [95]:
def evaluate_model(y_hat, y_test):
    classification = classification_report(y_hat, y_test)

    print(f"classification results : {classification}")

In [96]:
evaluate_model(pred, y_test)

classification results :               precision    recall  f1-score   support

           0       0.89      0.81      0.85       115
           1       0.70      0.81      0.75        64

    accuracy                           0.81       179
   macro avg       0.79      0.81      0.80       179
weighted avg       0.82      0.81      0.81       179

